In [1]:
import os
%cd /home/iir/work/ben/NCKU/IIR/SemEval2025_Task9
!pwd
import inference

/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9


/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9


/root/anaconda3/envs/augllms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_data_path = 'data/incidents_val.csv'

label_hazard_category = 'models/ST1/hazard_category/hazard-category_label_encoder.npy'
label_product_category = 'models/ST1/product_category/product-category_label_encoder.npy'

weight_hazard_category = 'models/ST1/hazard_category/best_st1_hazard_category_large_v2_250.pt' 
weight_product_category = 'models/ST1/product_category/best_st1_product_category_large_v2_250.pt'

bert_model_name = "microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract"
# bert_model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract"

# Run inference
predicted_hazard_category, predicted_probs = inference.main(test_data_path, label_hazard_category, weight_hazard_category, bert_model_name, max_len=300) 

predicted_product_category, predicted_probs = inference.main(test_data_path, label_product_category, weight_product_category, bert_model_name, max_len=300)

Predicting: 100%|██████████| 565/565 [00:06<00:00, 87.29it/s]


In [3]:
predicted_hazard_category

['biological',
 'foreign bodies',
 'foreign bodies',
 'allergens',
 'foreign bodies',
 'foreign bodies',
 'biological',
 'foreign bodies',
 'biological',
 'biological',
 'foreign bodies',
 'foreign bodies',
 'foreign bodies',
 'allergens',
 'allergens',
 'chemical',
 'allergens',
 'biological',
 'foreign bodies',
 'chemical',
 'biological',
 'allergens',
 'foreign bodies',
 'foreign bodies',
 'foreign bodies',
 'foreign bodies',
 'allergens',
 'biological',
 'allergens',
 'foreign bodies',
 'allergens',
 'biological',
 'allergens',
 'chemical',
 'fraud',
 'fraud',
 'foreign bodies',
 'foreign bodies',
 'foreign bodies',
 'foreign bodies',
 'allergens',
 'packaging defect',
 'chemical',
 'foreign bodies',
 'foreign bodies',
 'fraud',
 'foreign bodies',
 'allergens',
 'biological',
 'foreign bodies',
 'foreign bodies',
 'allergens',
 'fraud',
 'organoleptic aspects',
 'allergens',
 'foreign bodies',
 'fraud',
 'biological',
 'allergens',
 'foreign bodies',
 'foreign bodies',
 'foreign bo

In [4]:
predicted_product_category

['meat, egg and dairy products',
 'meat, egg and dairy products',
 'meat, egg and dairy products',
 'nuts, nut products and seeds',
 'meat, egg and dairy products',
 'cereals and bakery products',
 'meat, egg and dairy products',
 'alcoholic beverages',
 'meat, egg and dairy products',
 'seafood',
 'meat, egg and dairy products',
 'alcoholic beverages',
 'seafood',
 'soups, broths, sauces and condiments',
 'cereals and bakery products',
 'fats and oils',
 'cereals and bakery products',
 'meat, egg and dairy products',
 'cereals and bakery products',
 'non-alcoholic beverages',
 'meat, egg and dairy products',
 'cereals and bakery products',
 'prepared dishes and snacks',
 'cereals and bakery products',
 'cereals and bakery products',
 'cereals and bakery products',
 'dietetic foods, food supplements, fortified foods',
 'meat, egg and dairy products',
 'meat, egg and dairy products',
 'cereals and bakery products',
 'ices and desserts',
 'meat, egg and dairy products',
 'cereals and bak

In [5]:
import pandas as pd 
predictions = pd.DataFrame()
predictions["hazard-category"] = predicted_hazard_category
predictions["product-category"] = predicted_product_category 
predictions

,hazard-category,product-category
0,biological,"meat, egg and dairy products"
1,foreign bodies,"meat, egg and dairy products"
2,foreign bodies,"meat, egg and dairy products"
3,allergens,"nuts, nut products and seeds"
4,foreign bodies,"meat, egg and dairy products"
...,...,...
560,allergens,cereals and bakery products
561,allergens,"dietetic foods, food supplements, fortified foods"
562,foreign bodies,cereals and bakery products
563,allergens,cereals and bakery products


In [7]:
# save predictions to a new folder:
from shutil import make_archive
name_folder = "BERT_250_V5"

os.makedirs(f'submission/{name_folder}', exist_ok=True)
predictions.to_csv(f'submission/{name_folder}/submission.csv')

# zip the folder (zipfile can be directly uploaded to codalab):
make_archive(f'submission/{name_folder}', 'zip', f'submission/{name_folder}')

'/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9/submission/BERT_250_V5.zip'